In [ ]:
import os
import pandas as pd
from torch.utils.data import IterableDataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import requests
import json

#gene_list = list(pd.read_csv("./pam50.txt",sep="\t", header = None, index_col = False)[0])
gene_list = list(pd.read_excel("./pan_cancer.xlsx", header = None, index_col = False)[0])

In [ ]:
pd.read_excel("./pan_cancer.xlsx", header = None, index_col = False)

In [ ]:
len(gene_list)

In [ ]:
fields = [
    "file_name",
    "cases.submitter_id",
    "cases.samples.sample_type",
    "cases.disease_type",
    "file_id"
    ]

fields = ",".join(fields)

files_endpt = "https://api.gdc.cancer.gov/legacy/files"
# This set of filters is nested under an 'and' operator.
filters = {
    "op": "and",
    "content":[
        {
        "op": "in",
        "content":{
            "field": "cases.project.program.name",
            "value": ["TCGA"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "cases.project.primary_site",
            "value": ["Breast", "Lung"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "files.data_category",
            "value": ["Gene expression"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "files.data_type",
            "value": ["Gene expression quantification"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "files.experimental_strategy",
            "value": ["RNA-Seq"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "files.data_format",
            "value": ["TXT"]
            }
        }
    ]
}

# A POST is used, so the filter parameters can be passed directly as a Dict object.

params = {
    "filters": filters,
    "fields": fields,
    "format": "TSV",
    "size": "20000"
    }

# The parameters are passed to 'json' rather than 'params' in this case

In [ ]:
from io import StringIO
def manifest_loader(fields, filters,label):
    assert label in fields
    params = {
    "filters": filters,
    "fields": fields,
    "format": "TSV",
    "size": "20000"
    }

    response = requests.post(files_endpt, headers = {"Content-Type": "application/json"}, json = params)
    manifest = response.content

    data = StringIO(str(response.content,'utf-8')) 
    manifest=pd.read_csv(data, sep = '\t', index_col = False)
    
    return manifest, label

In [ ]:
logger = logging.getLogger()

# 로그의 출력 기준 설정
logger.setLevel(logging.INFO)

# log 출력 형식
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# log 출력

# log를 파일에 출력
file_handler = logging.FileHandler('analysis.log')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

In [ ]:
logger.info('test')

## Naive Implementation

In [28]:
import time, logging

logging.basicConfig(filename='analysis.log', encoding='utf-8', level=logging.DEBUG)

class GeneExpressionLoader(IterableDataset):
    def __init__(self,raw_data, label_column, gene_list, normalized = False ):
        super(GeneExpressionLoader).__init__()
        #raw_data, label_column = manifest_loader(fields, filters,label)
        ## 지정해준 fields, filter에 해당하는 파일들의 목록(manifest) 를 불러온다
        ## 어떤 coulmn을 label로 사용할 것인지 label_column을 통해 지정해주어야 한다. 
        raw_data.rename(lambda x : x.split('.')[-1], axis='columns', inplace = True)
        label_column = label_column.split('.')[-1]
        self.file_list = list(raw_data['file_id'])
        self.label_list = list(raw_data[label_column])
        self.label_dict = {}
        for idx, label in enumerate(set(self.label_list)):
            self.label_dict[label] = idx
        
        self.gene_list = gene_list
        self.normalized = normalized
        
            
       
    def __iter__(self):
         for filename,label in zip(self.file_list, self.label_list):
            logger.info('load_file')
            raw_data = pd.read_csv("./tcga/"+filename+"/"+os.listdir("./tcga/"+filename)[0], sep = '\t')
            # mount된 디렉토리에서 manifest 목록에 있는 file을 순차적으로 불러온다
            label_idx = self.label_dict[label]
            gene_column = ''
            count_column = ''
            if 'gene_id' in list(raw_data.columns):  gene_column = 'gene_id'
            elif 'gene' in list(raw_data.columns):  gene_column = 'gene'
                
            if not self.normalized:
                if 'raw_count' in list(raw_data.columns):  count_column = 'raw_count'
                elif 'raw_counts' in list(raw_data.columns):  count_column = 'raw_counts'
            #raw count도 파일마다 컬럼명이 다름.......        
            
            else : count_column = 'normalized_count'
            #print(raw_data.columns)
            try:
                raw_data[gene_column] = raw_data[gene_column].apply(lambda x: x.split('|')[0])
                #gene_id는 [symbol|PubMedID] (예 BRCA1|11331580) 의 형식으로 기재되어 있음
                #Symbol만을 남김
                raw_data = raw_data[raw_data[gene_column].isin(self.gene_list)]
                expression = raw_data[count_column].values
                logger.info('yield')
                yield (torch.tensor(expression) ,label_idx)
            except Exception as e:
                logger.info('skip')
                #print(e)
                continue

## Possible Caching Strategy

맨 처음으로 파일을 불러올 땐 약 20000여개의 유전자에 대한 발현량이 있는 파일을 통채로 불러온다.

gene list에 존재하는 유전자 : 1385개

모종의 이유로 gene_list에 존재하는 유전자 중 데이터에 존재하는 유전자 1346개

실제로 사용하는 데이터는 지정한 gene list에 있는 유전자(본 예시에서는 1346개)의 발현량이므로
이 1346개의 데이터만 캐싱하여 사용

In [31]:
from sklearn.model_selection import train_test_split
raw_data, label_column = manifest_loader(fields, filters,"cases.disease_type")
train_data, test_data = train_test_split(raw_data, test_size=0.33, random_state=42)

In [32]:
train_dataset = GeneExpressionLoader(train_data, label_column,gene_list)
test_dataset = GeneExpressionLoader(train_data, label_column,gene_list)
train_loader = DataLoader(train_dataset, batch_size = 10)
test_loader = DataLoader(test_dataset, batch_size = 10)

/home/mmsori/.conda/envs/nkhugh/lib/python3.8/site-packages/pandas/core/frame.py:4290: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


## Train-Test split 에 관해

하나의 manifest file 목록을 split하여 train과 test set을 생성함.

실제로는 data loader 안에서 유효한 데이터인지 판별하므로 실제 train과 test의 비중이 다름. 

In [ ]:
i = 0
for expression, label in train_loader:
    print(expression)
    i +=1
    if i == 10: break

### Model

모델 자체는 간단함.

간단한 모델 구조 안에서 complexity를 높이기 위해 층을 깊게 쌓음

In [ ]:
from torch import nn
import torch.nn.functional as F

In [ ]:
class DiseaseClassification(nn.Module):
    def __init__(self, input_dim ,hidden_dim, output_dim):
        
        super().__init__()
        self.fc1 = nn.Linear(input_dim,500)
        self.fc2 = nn.Linear(500,1000)
        self.fc3 = nn.Linear(1000,2000)
        self.fc4 = nn.Linear(2000,1000)
        self.fc5 = nn.Linear(1000,500)
        self.fc6 = nn.Linear(500, output_dim)
        
    def forward(self,x):
        x = x.float()
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        
        return x

In [ ]:
INPUT_SIZE = 1346
OUTPUT_SIZE = 3
HIDDEN_SIZE = 500
criterion = nn.CrossEntropyLoss()
model = DiseaseClassification(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)

In [ ]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) 
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for expression, label in iterator:
        
        logger.info('start_batch_train')
        optimizer.zero_grad()

        predictions = model(expression)

        loss = criterion(predictions, label)
                
        acc = categorical_accuracy(predictions, label)
        
        loss.backward()
        
        optimizer.step()
        logger.info('end_batch_train')
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for expression, label in iterator:

            predictions = model(expression)
        
            loss = criterion(predictions, label)  
            acc = categorical_accuracy(predictions, label)
        
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr = 0.02)

## 관찰

하나의 batch를 train하는 시간은 의외로 길지 않음(모델이 간단해서?)

오히려 dataloader streaming하는 overhead가 크다

In [ ]:
N_EPOCHS = 10
import time
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, train_loader, optimizer, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')